In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('td.textocurrienc > center > strong::text').get()
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[6]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['nombre']=nombre
        dipus.get(curp,{})['tipoElección']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract_first()
        dipus.get(curp,{})['entidad']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[1]
        dipus.get(curp,{})['distrito']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[2]
        # dc_dict['?????']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[3]
        dipus.get(curp,{})['correo']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[4]
        dipus.get(curp,{})['extensión']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[5]
        dipus.get(curp,{})['onomástico']=cumple
        supl=response.css('td.textocurri:nth-of-type(2)::text').extract()[6]
        dipus.get(curp,{})['suplente']=supl
        link_asis=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[0]
        link_votaciones=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[1]
        yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            if(votacion in por_votaciones.keys()):
                por_votaciones[votacion][curp]=posi
            else:
                por_votaciones[votacion] = {curp:posi}
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    return curp

In [4]:
dipus={}
curp = "h"
urlDipu = "http://sitl.diputados.gob.mx/LXIV_leg/"
urlCurr = "http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt="

por_votaciones = {}

terminaciones = []
for i in range(0,1):
    for j in range(0,1):
        for k in range(1,10):
            terminaciones.append(str(i)+str(j)+str(k))

# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-03-19 19:23:03 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-19 19:23:03 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-03-19 19:23:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-19 19:23:03 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-19 19:23:03 [scrapy.extensions.telnet] INFO: Telnet Password: 644c2ab139482cdb
2021-03-19 19:23:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-19 19:23:04 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-03-19 19:23:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/asistencias_por_pernplxiv.php?iddipt=6&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/asistencias_diputados_xperiodonplxiv.php?dipt=006)
2021-03-19 19:23:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=5&pert=4> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=005)
2021-03-19 19:23:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=6&pert=18> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=006)
2021-03-19 19:23:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=5&pert=3> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=005)


curp:MPÁ19oct85Nom: DECRETO POR EL QUE SE REFORMAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DE LAS ENTIDADES PARAESTATALES (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:MPÁ19oct85Nom: DECRETO POR EL QUE SE DECLARA EL TERCER MIÉRCOLES DE OCTUBRE COMO DÍA NACIONAL SOBRE LA RECONSTRUCCIÓN MAMARIA (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:MPÁ19oct85Nom: DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 51 DE LA LEY GENERAL DE VIDA SILVESTRE (EN LO GENERAL Y EN LO PARTICULAR CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA)  Posi: A favor
curp:MPÁ19oct85Nom: DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNIÓN DECLARA, EL ÚLTIMO MIÉRCOLES DEL MES DE MAYO DE CADA AÑO, COMO EL "DÍA NACIONAL DE LA ESCLEROSIS MÚLTIPLE" (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:MPÁ19oct85Nom: DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNIÓN DECLARA, EL DÍA 18 DE FEBRERO DE CADA AÑO, COMO EL "DÍA NACIONAL DEL SÍNDROME DE ASPERGER" (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
cu

2021-03-19 19:23:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=4&pert=14> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=004)
2021-03-19 19:23:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=4&pert=15> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=004)
2021-03-19 19:23:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=4&pert=18> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=004)



curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 5 DE LA LEY DE LA COMISIÓN NACIONAL DE LOS DERECHOS HUMANOS (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE EDUCACIÓN SUPERIOR Y SE ABROGA LA LEY PARA LA COORDINACIÓN DE LA EDUCACIÓN SUPERIOR (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  Posi: En contra
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE EDUCACIÓN SUPERIOR Y SE ABROGA LA LEY PARA LA COORDINACIÓN DE LA EDUCACIÓN SUPERIOR (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS, EN TÉRMINOS DEL DICTAMEN)  Posi: Ausente
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL PARA LA REGULACIÓN DEL CANNABIS, Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE SALUD Y DEL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  Posi: Abstención
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL PARA LA REGULAC

...

curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 201 Y 205 DE LA LEY DEL SEGURO SOCIAL (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS, CON LAS MODIFICACIONES ACEPTADAS)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 456 Y 457 DE LA LEY GENERAL DE INSTITUCIONES Y PROCEDIMIENTOS ELECTORALES (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  Posi: Ausente
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 456 Y 457 DE LA LEY GENERAL DE INSTITUCIONES Y PROCEDIMIENTOS ELECTORALES (EN LO PARTICULAR EL ARTÍCULO 457 RESERVADO, CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA)  Posi: Ausente
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMA LA FRACCIÓN III DEL ARTÍCULO 113, Y SE ADICIONA UN ARTÍCULO 113 BIS AL CÓDIGO FISCAL DE LA FEDERACIÓN (EN LO GENERAL Y EN LO PARTICULAR)  Posi: Ausente
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL PARA PREVENIR Y SANCIONAR LOS DE

curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY DEL DIARIO OFICIAL DE LA FEDERACIÓN Y GACETAS GUBERNAMENTALES (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 104 DE LA LEY ORGÁNICA DE CONGRESO GENERAL DE LOS ESTADOS UNIDOS MEXICANOS (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 2o Y 22 DE LA LEY DE DISCIPLINA FINANCIERA DE LAS ENTIDADES FEDERATIVAS Y LOS MUNICIPIOS (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE ADICIONA UN SEGUNDO PÁRRAFO AL ARTÍCULO 74 DE LA LEY GENERAL DE ASENTAMIENTOS HUMANOS, ORDENAMIENTO TERRITORIAL Y DESARROLLO URBANO (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 211 BIS 1 DEL CÓDIGO PENAL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR)  Posi: A favor
curp:ARL1

2021-03-19 19:23:50 [scrapy.core.engine] INFO: Closing spider (finished)


curp:ARL10abr72Nom: DECRETO POR EL QUE SE REFORMA EL PRIMER PÁRRAFO DEL ARTÍCULO 40 Y SE ADICIONA UN SEGUNDO PÁRRAFO AL ARTÍCULO 44 BIS 4, DE LA LEY DE INSTITUCIONES DE CRÉDITO (EN LO GENERAL Y EN LO PARTICULAR)  Posi: Ausente
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL DE AUSTERIDAD REPUBLICANA DE ESTADO; Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, Y DE LA LEY FEDERAL DE PRESUPUESTO Y RESPONSABILIDAD HACENDARIA (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  Posi: A favor
curp:ARL10abr72Nom: DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL DE AUSTERIDAD REPUBLICANA DE ESTADO; Y SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, Y DE LA LEY FEDERAL DE PRESUPUESTO Y RESPONSABILIDAD HACENDARIA (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS,EN SUS TÉRMINOS)  Posi: A favor
curp:ARL10abr72Nom: ACUERDO DE LA JUNTA DE COORDINACIÓN POLÍTICA RELATIVO A LA D

2021-03-19 19:23:50 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 102634,
 'downloader/request_count': 222,
 'downloader/request_method_count/GET': 222,
 'downloader/response_bytes': 6757177,
 'downloader/response_count': 222,
 'downloader/response_status_count/200': 222,
 'elapsed_time_seconds': 46.294219,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 3, 20, 1, 23, 50, 572492),
 'log_count/DEBUG': 222,
 'log_count/INFO': 10,
 'request_depth_max': 2,
 'response_received_count': 222,
 'scheduler/dequeued': 222,
 'scheduler/dequeued/memory': 222,
 'scheduler/enqueued': 222,
 'scheduler/enqueued/memory': 222,
 'start_time': datetime.datetime(2021, 3, 20, 1, 23, 4, 278273)}
2021-03-19 19:23:50 [scrapy.core.engine] INFO: Spider closed (finished)


In [5]:
for key, value in dipus.items():
    print('\n',key, ' : \n\n', value)


 ARL10abr72  : 

 {'nombre': 'Dip. Armando Reyes Ledesma', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Baja California ', 'distrito': '3 ', 'correo': 'armando.reyes@diputados.gob.mx ', 'extensión': '62030 ', 'onomástico': '10-abril-1972 ', 'suplente': 'Christian Hiram Dunn Fitch\n\t\t\t\t\t\n\n\n ', 'Asistencias': 135, 'votos': 589}

 MPÁ19oct85  : 

 {'nombre': 'Dip. Marina del Pilar Ávila Olmeda (LICENCIA)', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Baja California ', 'distrito': '2 ', 'correo': 'marina.avila@diputados.gob.mx ', 'extensión': '61372 ', 'onomástico': '19-octubre-1985 ', 'suplente': 'Martha Lizeth Noriega Galaz\n\t\t\t\t\t\t\t\t\t\t', 'votos': 48, 'Asistencias': 43}

 MIM17jun48  : 

 {'nombre': 'Dip. Mario Ismael Moreno Gil', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Baja California ', 'distrito': '5 ', 'correo': 'ismael.moreno@diputados.gob.mx ', 'extensión': '61327 ', 'onomástico': '17-junio-1948 ', 'suplente': 'Luis Javier Algorri Franco\n\t\t\t\t\t

In [13]:
for key, value in por_votaciones.items():
    print('\n',key,': ')
    for curp, posi in value.items():
        print('\n',curp, ': ', posi)


 DECRETO POR EL QUE SE REFORMAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DE LAS ENTIDADES PARAESTATALES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MPÁ19oct85 :  A favor

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  En contra

 FJL01jul79 :  En contra

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE DECLARA EL TERCER MIÉRCOLES DE OCTUBRE COMO DÍA NACIONAL SOBRE LA RECONSTRUCCIÓN MAMARIA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 MPÁ19oct85 :  A favor

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 51 DE LA LEY GENERAL DE VIDA SILVESTRE (EN LO GENERAL Y EN LO PARTICULAR CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA)  : 

 MPÁ19oct85 :  A favor

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor



 JSM01jun51 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONA LOS ARTÍCULOS 2, 29 Y 32 DE LA LEY GENERAL PARA LA INCLUSIÓN DE LAS PERSONAS CON DISCAPACIDAD (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ARL10abr72 :  A favor

 SIA22sep64 :  A favor

 JSM01jun51 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 DECRETO, POR EL QUE SE ADICIONA EL SEGUNDO PÁRRAFO DEL ARTÍCULO 38 DE LA LEY DE OBRAS PÚBLICAS Y SERVICIOS RELACIONADOS CON LAS MISMAS (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ARL10abr72 :  A favor

 SIA22sep64 :  A favor

 JSM01jun51 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES DE LAS LEYES DEL INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO; FEDERAL DE LOS TRABAJADORES AL SERVICIO DEL ESTADO, REGLAMENTARIA DEL APARTADO B DEL ARTÍCULO 123 CONSTITUCIONAL; 

 ELT15mar77 :  En contra

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE REFORMA Y ADICIONA EL ARTÍCULO 167 DEL CÓDIGO NACIONAL DE PROCEDIMIENTOS PENALES; Y SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LAS LEYES GENERAL EN MATERIA DE DELITOS ELECTORALES, FEDERAL PARA PREVENIR Y SANCIONAR LOS DELITOS COMETIDOS EN MATERIA DE HIDROCARBUROS, Y FEDERAL DE ARMAS DE FUEGO Y EXPLOSIVOS, ASÍ COMO DEL CÓDIGO PENAL FEDERAL, Y DE LAS LEYES GENERAL DE SALUD, FEDERAL CONTRA LA DELINCUENCIA ORGANIZADA, Y DE VÍAS GENERALES DE COMUNICACIÓN (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 SIA22sep64 :  A favor

 JSM01jun51 :  A favor

 FJL01jul79 :  Ausente

 MEG29ene77 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE REFORMA Y ADICIONA EL ARTÍCULO 167 DEL CÓDIGO NACIONAL DE PROCEDIMIENTOS PENALES; Y SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LAS LEYES GENERAL EN MATERIA DE

 ELT15mar77 :  A favor

 MIM17jun48 :  Ausente

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY DE DESARROLLO RURAL SUSTENTABLE, DE LA LEY GENERAL DE DESARROLLO SOCIAL, DE LA LEY ORGÁNICA DE LA ADMINISTRACIÓN PÚBLICA FEDERAL Y DE LA LEY FEDERAL DE FOMENTO A LAS ACTIVIDADES REALIZADAS POR ORGANIZACIONES DE LA SOCIEDAD CIVIL, EN MATERIA DE SEGURIDAD ALIMENTARIA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 SIA22sep64 :  A favor

 JSM01jun51 :  A favor

 FJL01jul79 :  A favor

 MEG29ene77 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  Ausente

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE POBLACIÓN, PARA ESTABLECER UN DOCUMENTO PROVISIONAL DE IDENTIDAD A LOS MIGRANTES MEXICANOS EN SITUACIÓN DE RETORNO (EN LO GENERAL Y EN LO PARTICULAR)  : 

 SIA22sep64 :  A favor

 JSM01jun51 :  Ausente

 FJL01jul79 :  A favor

 MEG29ene77 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  Ausente

 

 JSM01jun51 :  Ausente

 FJL01jul79 :  Ausente

 MEG29ene77 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY PARA LA PROTECCIÓN DE PERSONAS DEFENSORAS DE DERECHOS HUMANOS Y PERIODISTAS (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA)  : 

 SIA22sep64 :  A favor

 JSM01jun51 :  Ausente

 FJL01jul79 :  En contra

 MEG29ene77 :  En contra

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE POBLACIÓN (EN LO GENERAL Y EN LO PARTICULAR)  : 

 SIA22sep64 :  A favor

 JSM01jun51 :  Ausente

 FJL01jul79 :  A favor

 MEG29ene77 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  Ausente

 ARL10abr72 :  Ausente

 DECRETO POR EL QUE SE ADICIONA UNA FRACCIÓN IV AL INCISO A) Y SE DEROGA LA FRACCIÓN IV DEL INCISO B) DEL ARTÍCULO 298 DE LA LEY FEDERAL DE TELECOMUNICACIONES

...

 JJC07oct67 :  Ausente

 DECRETO POR EL QUE SE REFORMA LA FRACCIÓN II DEL ARTÍCULO 73 DE LA LEY GENERAL DE TRANSPARENCIA Y ACCESO A LA INFORMACIÓN PÚBLICA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  Ausente

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 JJC07oct67 :  Ausente

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE SALUD, Y DE LA LEY DE LOS INSTITUTOS NACIONALES DE SALUD (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  En contra

 FJL01jul79 :  Ausente

 ELT15mar77 :  Ausente

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 JJC07oct67 :  Ausente

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY GENERAL DE SALUD, Y DE LA LEY DE LOS INSTITUTOS NACIONALES DE SALUD (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS,

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 JJC07oct67 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY DE LA GUARDIA NACIONAL (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 JJC07oct67 :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY DE LA GUARDIA NACIONAL (EN LO PARTICULAR LOS ARTÍCULOS 8, 14, 86, 93 Y DÉCIMO TERCERO TRANSITORIO RESERVADOS, EN SUS TÉRMINOS)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 ARL10abr72 :  Ausente

 JJC07oct67 :  A favor

 DECRETO POR SE EXPIDE LA LEY NACIONAL SOBRE EL USO DE LA FUERZA (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A f

 ELT15mar77 :  En contra

 MIM17jun48 :  A favor

 MPÁ19oct85 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 LEY DE INGRESOS DE LA FEDERACIÓN PARA EL EJERCICIO FISCAL DE 2019 (EN LO PARTICULAR LOS ARTICULOS RESERVADOS 1, 27; SÉPTIMO, DÉCIMO, DÉCIMO SEGUNDO TRANSITORIOS CON LAS MODIFICACIONES ACEPTADAS POR LA ASAMBLEA Y ADICIÓN DE UN ARTÍCULO TRANSITORIO)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  En contra

 FJL01jul79 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 MPÁ19oct85 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 LEY DE INGRESOS DE LA FEDERACIÓN PARA EL EJERCICIO FISCAL DE 2019 (EN LO PARTICULAR LOS ARTICULOS RESERVADOS 2, 11, 12, 16, 21, 23, 24 Y 25 EN TÉRMINOS DEL DICTAMEN)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  En contra

 FJL01jul79 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 MPÁ19oct85 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL 

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMA LA FRACCIÓN VI DEL ARTÍCULO 112 DE LA LEY GENERAL DEL EQUILIBRIO ECOLÓGICO Y LA PROTECCIÓN AL AMBIENTE (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  Ausente

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  Ausente

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 70 Y 70 BIS DE LA LEY DE MIGRACIÓN (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 109 Y 140 DE LA LEY DE MIGRACIÓN (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSOS ARTÍCULOS DE LA LEY FEDERAL DE LOS TRABAJADORES AL SERVICIO DEL ESTADO, REGLAMENTARIA DEL APARTADO B) DEL ARTÍCULO 123 CONSTITUCIONAL (EN LO PARTICULAR EL ARTÍCULO 78 RESERVADO, EN SUS TÉRMINOS)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE ADICIONA UN PÁRRAFO AL ARTÍCULO 26 DE LA LEY GENERAL DE DESARROLLO SOCIAL (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor

 SIA22sep64 :  A favor

 MEG29ene77 :  A favor

 FJL01jul79 :  A favor

 ELT15mar77 :  A favor

 MIM17jun48 :  A favor

 JJC07oct67 :  A favor

 ARL10abr72 :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEROGAN DIVERSAS DISPOSICIONES DE LA LEY DE PREMIOS, ESTÍMULOS Y RECOMPENSAS CIVILES (EN LO GENERAL Y EN LO PARTICULAR)  : 

 JSM01jun51 :  A favor


In [ ]:
import json

with open('votos.json', 'w') as fp:
    json.dump(por_votaciones, fp)

In [11]:
string = "\t\t\tHello, how are you\n\n"
print("Old String:")
print("'" + string + "'")

Old String:
'			Hello, how are you

'


In [12]:
string = string.strip('\n')
string = string.strip('\t')
print("New String:")
print("'" + string + "'")

New String:
'Hello, how are you'


In [35]:
mydic={"Ab":{"votaciones":{"Votación 1":"yes","Votación 2":"no","Votación 3":"tal vez"}},"Bc":{"votaciones":{"Votación 1":"no","Votación 2":"yes","Votación 3":"chance"}}}
mydic2={"Votación 1":{"Ab":"yes","Bc":"no","Cd":"chance"},"Votación 2":{"Ab":"npi","Bc":"bueno","Cd":"en una de ésas"}}

In [32]:
list_of_dicts = [{"a": 1, "b": 2, "c": 3}, {"a": 4, "b": 5, "c": 6}, {"a": 7, "b": 8, "c": 9}]

a_key = "Votación 1"

for key, value in mydic.items():
    print("Clave " + key)
    print(value["votaciones"]["Votación 1"])
    #values_of_key = [a_dict[a_key] for a_dict in list_of_dicts]

Clave Ab
yes
Clave Bc
no


In [33]:
print(mydic["Ab"]["votaciones"]["Votación 1"])

yes


In [34]:
mydic["Ab"]["votaciones"]["Votación 4"] = "maybe"
mydic

{'Ab': {'votaciones': {'Votación 1': 'yes',
   'Votación 2': 'no',
   'Votación 3': 'tal vez',
   'Votación 4': 'maybe'}},
 'Bc': {'votaciones': {'Votación 1': 'no',
   'Votación 2': 'yes',
   'Votación 3': 'chance'}}}

In [12]:

#mydic={'DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES A LA LEY GENERAL DE ACCESO DE LAS MUJERES A UNA VIDA LIBRE DE VIOLENCIA (EN LO GENERAL Y EN LO PARTICULAR)': {'Curp1': 'simón'}}
#mydic["DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES A LA LEY GENERAL DE ACCESO DE LAS MUJERES A UNA VIDA LIBRE DE VIOLENCIA (EN LO GENERAL Y EN LO PARTICULAR)"] = {}
mydic={}
if("votacion" in mydic.keys()):
    mydic["votacion"]["Curp2"]="posi"
else:
    mydic["votacion"] = {"Curp1":"posi"}

mydic

{'votacion': {'Curp1': 'posi'}}

In [36]:
mydic2["Votación 1"]["De"]="porsupollo"
mydic2

{'Votación 1': {'Ab': 'yes', 'Bc': 'no', 'Cd': 'chance', 'De': 'porsupollo'},
 'Votación 2': {'Ab': 'npi', 'Bc': 'bueno', 'Cd': 'en una de ésas'}}

In [ ]:
if(por_votaciones[votacion] == {}):
                por_votaciones[votacion] = {curp:posi}
            else:
                por_votaciones[votacion][curp]=posi